In [46]:
import utils
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
import pandas as pd
import numpy as np
import os
import math

In [47]:
queries = utils.load_queries()

tokenizer = RegexpTokenizer(
    r'(?:[A-Za-z]\.)+'
    r'|[A-Za-z]+[\-@]\d+(?:\.\d+)?'
    r'|\d+(?:[\.\,\-]\d+)*%?'
    r'|[A-Za-z]+'
)
nltk.download('stopwords', quiet=True)
stop_words = set(stopwords.words('english'))

In [48]:
def query_cleaning(query):
  # Tokenize and clean query
  tokens = tokenizer.tokenize(query)
  clean_query = [word.lower() for word in tokens if word.lower() not in stop_words]
  
  Porter = PorterStemmer()
  TermsPorter = [Porter.stem(t) for t in clean_query]
  
  return TermsPorter

def calc_weight_Word_query(query, word):
  if word in query:
    return 1
  return 0

In [49]:
clean_queries =[]
for q in queries:
    clean_queries.append(query_cleaning(q))

In [50]:
data = pd.read_csv(
    "Docterm_matrix.txt",
    sep=r"\s+", header=None,
    names=["Document", "Term", "Frequency", "Weight"]
)

data = data.iloc[1:].reset_index(drop=True)

In [51]:
print("📄 Aperçu du DataFrame original :")
print(data.head())

📄 Aperçu du DataFrame original :
  Document      Term Frequency    Weight
0    doc_1      acid         1  0.126638
1    doc_1    appear         1  0.107622
2    doc_1  coeffici         2  0.515906
3    doc_1   collect         1  0.217908
4    doc_1    correl         4  0.648133


In [52]:
data["Weight"] = pd.to_numeric(data["Weight"], errors='coerce')

In [53]:
data["Frequency"] = pd.to_numeric(data["Frequency"], errors='coerce')

In [54]:
df_agg = data.groupby(["Document", "Term"], as_index=False)["Weight"].mean()

In [55]:
M_Weight= df_agg.pivot(index="Term", columns="Document", values="Weight").fillna(0)

In [56]:
df_freq_agg = data.groupby(["Document", "Term"], as_index=False)["Frequency"].sum()

M_Freq = df_freq_agg.pivot(index="Term", columns="Document", values="Frequency").fillna(0)

In [57]:
print("\n Matrice terme-document (W) :")
print(M_Weight)


 Matrice terme-document (W) :
Document  doc_1  doc_10  doc_100  doc_1000  doc_1001  doc_1002  doc_1003  \
Term                                                                       
0           0.0     0.0      0.0       0.0       0.0       0.0       0.0   
0-15        0.0     0.0      0.0       0.0       0.0       0.0       0.0   
0.001       0.0     0.0      0.0       0.0       0.0       0.0       0.0   
0.003       0.0     0.0      0.0       0.0       0.0       0.0       0.0   
0.004       0.0     0.0      0.0       0.0       0.0       0.0       0.0   
...         ...     ...      ...       ...       ...       ...       ...   
zophren     0.0     0.0      0.0       0.0       0.0       0.0       0.0   
zora        0.0     0.0      0.0       0.0       0.0       0.0       0.0   
zuelzer     0.0     0.0      0.0       0.0       0.0       0.0       0.0   
zygou       0.0     0.0      0.0       0.0       0.0       0.0       0.0   
zyme        0.0     0.0      0.0       0.0       0.0     

In [58]:
U, S, VT = np.linalg.svd(M_Weight, full_matrices=False)


In [59]:
k = 100

U_k = U[:, :k]      
S_k = S[:k]         
VT_k = VT[:k,:]     

In [60]:
S_k_inv = np.diag(1 / S_k)   

In [61]:
M = np.dot(U_k, S_k_inv)

In [62]:
binary_queries = []
words = M_Weight.index

for q in clean_queries:
    binary_q = [0] * len(words)
    for i in range(len(words)):
        binary_q[i] = calc_weight_Word_query(q , words[i])
        
    binary_queries.append(binary_q)

In [63]:
print(binary_queries)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [64]:
Qnew = []
sim = []

for q in binary_queries:
    # la projection de Q en utilisant la matrice M
    qnew = q @ M
    Qnew.append(qnew)
    
    # calcul de similarities   𝑄𝑛𝑒𝑤.S.S.𝐷 
    sim.append(qnew @ (np.diag(S_k) @ np.diag(S_k)) @ VT_k)

In [65]:
sim_df = pd.DataFrame(sim, 
                      columns=M_Weight.columns,  # noms des documents
                      index=[f"Query {i+1}" for i in range(len(clean_queries))])

In [66]:
sim_df_rounded = sim_df.round(3)

In [67]:
print("📊 Similarité requêtes-documents :")
print(sim_df_rounded)

📊 Similarité requêtes-documents :
Document  doc_1  doc_10  doc_100  doc_1000  doc_1001  doc_1002  doc_1003  \
Query 1  -0.025  -0.146    0.047     0.026     0.042    -0.010     0.240   
Query 2   0.116   0.515    0.288     0.229     0.172     0.181     0.688   
Query 3  -0.016  -0.043   -0.031     0.048     0.069     0.077     1.181   
Query 4   0.003  -0.025   -0.055     0.153     0.127     0.260     1.295   
Query 5   1.304   2.002    0.060     0.038     0.044    -0.045    -0.062   
Query 6   0.115   0.039    0.290    -0.055     0.121     0.250    -0.033   
Query 7  -0.003   0.120    0.224     0.566     0.764     0.381    -0.153   
Query 8   0.023   0.139    0.236     0.230     0.483     0.148     0.523   
Query 9   0.010  -0.105    0.287     0.395     0.461     0.347    -0.167   
Query 10 -0.008  -0.014   -0.004     0.046     0.059     0.002     0.022   
Query 11  0.089   0.117   -0.003     0.179     0.336     0.161    -0.004   
Query 12 -0.035  -0.112    0.152     0.286     0.477  

In [68]:
for i, row in sim_df.iterrows():
    print(f"\nTop documents pour {i} :")
    print(row.sort_values(ascending=False))


Top documents pour Query 1 :
Document
doc_13     2.168192
doc_181    1.984439
doc_169    1.970156
doc_510    1.883662
doc_142    1.850400
             ...   
doc_416   -0.242381
doc_320   -0.247609
doc_855   -0.253993
doc_786   -0.262018
doc_22    -0.282020
Name: Query 1, Length: 1033, dtype: float64

Top documents pour Query 2 :
Document
doc_296    3.269373
doc_724    2.642532
doc_187    2.136294
doc_297    2.004338
doc_292    1.864523
             ...   
doc_621   -0.261414
doc_885   -0.262253
doc_526   -0.322637
doc_814   -0.372081
doc_136   -0.544944
Name: Query 2, Length: 1033, dtype: float64

Top documents pour Query 3 :
Document
doc_286     1.961518
doc_69      1.756563
doc_282     1.631304
doc_287     1.532201
doc_233     1.400639
              ...   
doc_879    -0.140244
doc_725    -0.150293
doc_201    -0.154864
doc_978    -0.155131
doc_1005   -0.158116
Name: Query 3, Length: 1033, dtype: float64

Top documents pour Query 4 :
Document
doc_468     1.407768
doc_1003    1.295053

# Classic BIR – Without learning data -Smooth formula

In [69]:
mat_bin = M_Weight.map(lambda x: 1 if x > 0 else 0)


In [70]:
DocNum = len(mat_bin.columns)

In [71]:
def load_relevance():
  relevance = {}
  relevance_folder = "./Relevances/"  # adjust path

  for filename in os.listdir(relevance_folder):
      if filename.endswith(".txt"):
          query_name = filename.replace(".txt", "")  # e.g., "Query1"
          query_name = f"{query_name}"
          file_path = os.path.join(relevance_folder, filename)
          with open(file_path, "r") as f:
              docs = [f"doc_{line.strip()}" for line in f if line.strip()]
          relevance[query_name] = docs
          
  return relevance

In [72]:
rels = load_relevance()

In [73]:
length_all = mat_bin.sum() 

In [74]:
average_length = length_all.mean()

In [75]:
k1 = 1.2
b=0.75

In [76]:
RSV_all1 = {}
RSV_all2 = {}
RSV_all3 = {}
RSV_all4 = {}
RSV_BM25_all={}

for i, q in enumerate(clean_queries):
    # Identifiant de la requête pour accéder au dictionnaire (ex: "1", "2", ...)
    query_id = str(i + 1)
    
    # 1. Récupérer la liste des docs pertinents pour cette requête
    # On utilise .get() avec une liste vide par défaut pour éviter les erreurs
    relevant_docs = rels.get(query_id, [])
    
    # 2. Définir R (nombre total de documents pertinents pour cette requête)
    R = len(relevant_docs)
    
    RSV1 = {doc: 0 for doc in mat_bin.columns}
    RSV2 = {doc: 0 for doc in mat_bin.columns}
    RSV3 = {doc: 0 for doc in mat_bin.columns}
    RSV4 = {doc: 0 for doc in mat_bin.columns}
    RSV_BM25 = {doc: 0 for doc in mat_bin.columns}

    for term in q:
        if term in mat_bin.index:
            ni = mat_bin.loc[term].sum()  # nombre total de docs contenant le terme
            
            # 3. Calculer ri : nombre de docs pertinents contenant le terme
            # On ne somme que sur les colonnes présentes dans 'relevant_docs'
            if R > 0:
                # On s'assure que les docs de 'relevant_docs' existent bien dans les colonnes de mat_bin
                existing_rel_docs = [d for d in relevant_docs if d in mat_bin.columns]
                ri = mat_bin.loc[term, existing_rel_docs].sum()
            else:
                ri = 0

            for doc in mat_bin.columns:
                dl = mat_bin[doc].sum()  
                
                if mat_bin.loc[term, doc] == 1:  # terme présent dans le document actuel
                    wi = M_Weight.loc[term, doc]
                    tfi = M_Freq.loc[term, doc]
                    w_iq = calc_weight_Word_query(q, term)
                    
                    # Formules avec R et ri dynamiques
                    RSV1[doc] += math.log10( (DocNum - ni + 0.5) / (ni + 0.5) )
                    RSV2[doc] += math.log10( ((ri+0.5) / (R-ri+0.5)) / ((ni-ri+0.5) / (DocNum-ni-R+ri+0.5)))
                    RSV3[doc] += wi * w_iq * math.log10( (DocNum - ni + 0.5) / (ni + 0.5) )
                    RSV4[doc] += wi * w_iq * math.log10( ((ri+0.5) / (R-ri+0.5)) / ((ni-ri+0.5) / (DocNum-ni-R+ri+0.5)))
                    RSV_BM25[doc] += math.log10( (DocNum - ni + 0.5) / (ni + 0.5) ) * (( (k1+1)*tfi ) / ( (k1 * ((1-b) + (b* (dl/average_length) ))) + tfi))
                    
    RSV_all1[f"Query {i+1}"] = RSV1
    RSV_all2[f"Query {i+1}"] = RSV2
    RSV_all3[f"Query {i+1}"] = RSV3
    RSV_all4[f"Query {i+1}"] = RSV4
    RSV_BM25_all[f"Query {i+1}"] = RSV_BM25

In [77]:
print("-----Classic BIR – Without learning data - Smooth formula")

RSV_df = pd.DataFrame(RSV_all1)
print(RSV_df)

# Pour chaque document, trier les requêtes selon leur score (ordre décroissant)
for doc_id in RSV_df.columns:
    print(f"\nTop requêtes pour le document {doc_id} :")
    print(RSV_df[doc_id].sort_values(ascending=False))

print("-----Classic BIR – With learning data - Smooth formula")

RSV_df = pd.DataFrame(RSV_all2)
print(RSV_df)

for doc_id in RSV_df.columns:
    print(f"\nTop requêtes pour le document {doc_id} :")
    print(RSV_df[doc_id].sort_values(ascending=False))

    
    
print("-----Extended BIR  – Without learning data -Smooth formula")

RSV_df = pd.DataFrame(RSV_all3)
print(RSV_df)

for doc_id in RSV_df.columns:
    print(f"\nTop requêtes pour le document {doc_id} :")
    print(RSV_df[doc_id].sort_values(ascending=False))

   
print("-----Extended BIR  – With learning data -Smooth formula")

RSV_df = pd.DataFrame(RSV_all4)
print(RSV_df)

for doc_id in RSV_df.columns:
    print(f"\nTop requêtes pour le document {doc_id} :")
    print(RSV_df[doc_id].sort_values(ascending=False))



print("-----BM25-----")

RSV_df = pd.DataFrame(RSV_BM25_all)
print(RSV_df)

for doc_id in RSV_df.columns:
    print(f"\nTop requêtes pour le document {doc_id} :")
    print(RSV_df[doc_id].sort_values(ascending=False))

-----Classic BIR – Without learning data - Smooth formula
           Query 1   Query 2  Query 3  Query 4  Query 5   Query 6   Query 7  \
doc_1     0.000000  0.000000      0.0      0.0  5.57851  0.000000  0.000000   
doc_10    0.000000  0.772057      0.0      0.0  5.57851  0.000000  0.000000   
doc_100   1.097477  0.000000      0.0      0.0  0.00000  0.875657  0.674726   
doc_1000  0.000000  1.424446      0.0      0.0  0.00000  0.000000  2.851519   
doc_1001  1.097477  0.875657      0.0      0.0  0.00000  0.000000  5.649235   
...            ...       ...      ...      ...      ...       ...       ...   
doc_995   0.000000  2.509125      0.0      0.0  0.00000  0.000000  3.130369   
doc_996   0.000000  0.000000      0.0      0.0  0.00000  0.000000  3.130369   
doc_997   0.000000  0.875657      0.0      0.0  0.00000  0.000000  1.097477   
doc_998   0.000000  0.000000      0.0      0.0  0.00000  0.000000  0.000000   
doc_999   1.097477  0.000000      0.0      0.0  0.00000  0.000000  1.0974

In [78]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve, average_precision_score

In [79]:
def overall_precision(ranked_docs, relevant_docs):
    tp = sum([1 for doc in ranked_docs if doc in relevant_docs])
    fp = len(ranked_docs) - tp
    return tp / (tp + fp) if (tp + fp) > 0 else 0

def overall_recall(ranked_docs, relevant_docs):
    tp = sum([1 for doc in ranked_docs if doc in relevant_docs])
    fn = len(relevant_docs) - tp
    return tp / (tp + fn) if (tp + fn) > 0 else 0

def overall_f1(ranked_docs, relevant_docs):
    p = overall_precision(ranked_docs, relevant_docs)
    r = overall_recall(ranked_docs, relevant_docs)
    return 2 * p * r / (p + r) if (p + r) > 0 else 0

# -----------------------------
# Top-K Metrics
# -----------------------------
def precision_at_k(ranked_docs, relevant_docs, k):
    ranked_docs = ranked_docs[:k]
    relevant_count = sum([1 for doc in ranked_docs if doc in relevant_docs])
    return relevant_count / k

# -----------------------------
# Overall Metrics
# -----------------------------
def r_precision(ranked_docs, relevant_docs):
    R = len(relevant_docs)
    return precision_at_k(ranked_docs, relevant_docs, R)

def reciprocal_rank(ranked_docs, relevant_docs):
    for idx, doc in enumerate(ranked_docs, 1):
        if doc in relevant_docs:
            return 1 / idx
    return 0

def dcg_at_k(ranked_docs, relevant_docs, k):
    dcg = 0
    for i, doc in enumerate(ranked_docs[:k], 1):
        if doc in relevant_docs:
            dcg += 1 / np.log2(i + 1)
    return dcg

def ndcg_at_k(ranked_docs, relevant_docs, k):
    ideal_dcg = dcg_at_k(sorted(relevant_docs, key=lambda x: 1, reverse=True), relevant_docs, k)
    actual_dcg = dcg_at_k(ranked_docs, relevant_docs, k)
    return actual_dcg / ideal_dcg if ideal_dcg > 0 else 0

# -----------------------------
# Precision-Recall Curve
# -----------------------------
def precision_recall_curve(ranked_docs, relevant_docs):
    precisions, recalls = [], []
    n_relevant = len(relevant_docs)
    retrieved_relevant = 0

    for i, doc in enumerate(ranked_docs, 1):
        if doc in relevant_docs:
            retrieved_relevant += 1
        precisions.append(retrieved_relevant / i)
        recalls.append(retrieved_relevant / n_relevant if n_relevant else 0)
    return recalls, precisions

# -----------------------------
# Interpolated Precision-Recall Curve
# -----------------------------
def interpolated_precision_recall_curve(ranked_docs, relevant_docs):
    recalls, precisions = precision_recall_curve(ranked_docs, relevant_docs)
    interpolated_precisions = []
    for r in np.linspace(0, 1, 11):  # standard 11-point interpolation
        p = max([prec for rec, prec in zip(recalls, precisions) if rec >= r] + [0])
        interpolated_precisions.append(p)
    return np.linspace(0, 1, 11), interpolated_precisions

# -----------------------------
# Average Precision (AP) & MAP
# -----------------------------
def average_precision(ranked_docs, relevant_docs):
    precisions = []
    retrieved_relevant = 0
    for i, doc in enumerate(ranked_docs, 1):
        if doc in relevant_docs:
            retrieved_relevant += 1
            precisions.append(retrieved_relevant / i)
    return np.mean(precisions) if precisions else 0

def mean_average_precision(all_ranked_docs, all_relevant_docs):
    ap_scores = []
    for q in all_ranked_docs:
        ap_scores.append(average_precision(all_ranked_docs[q], all_relevant_docs[q]))
    return np.mean(ap_scores)

# -----------------------------
# Interpolated MAP
# -----------------------------
def interpolated_average_precision(ranked_docs, relevant_docs):
    recalls, precisions = precision_recall_curve(ranked_docs, relevant_docs)
    interpolated_precisions = []
    for r in np.linspace(0, 1, 11):
        p = max([prec for rec, prec in zip(recalls, precisions) if rec >= r] + [0])
        interpolated_precisions.append(p)
    return np.mean(interpolated_precisions)

def mean_interpolated_average_precision(all_ranked_docs, all_relevant_docs):
    scores = []
    for q in all_ranked_docs:
        scores.append(interpolated_average_precision(all_ranked_docs[q], all_relevant_docs[q]))
    return np.mean(scores)

# -----------------------------
# Gain (%) for nDCG@20
# -----------------------------
def gain_percent(all_ranked_docs, all_relevant_docs, k=20):
    gains = []
    for q in list(all_ranked_docs.keys())[:10]:  # first 10 queries
        ndcg = ndcg_at_k(all_ranked_docs[q], all_relevant_docs[q], k)
        gains.append(ndcg)
    return np.mean(gains) * 100

In [80]:
def get_ranked_docs(all_rsv_results):
    ranked_results = {}
    for q_name, data in all_rsv_results.items():
        # Si c'est un dictionnaire {doc: score}, on trie par score décroissant
        if isinstance(data, dict):
            sorted_docs = sorted(data.items(), key=lambda x: x[1], reverse=True)
            ranked_results[q_name] = [doc for doc, score in sorted_docs]
        # Si c'est déjà une liste [doc1, doc2], on la laisse telle quelle
        elif isinstance(data, list):
            ranked_results[q_name] = data
    return ranked_results

In [81]:
formatted_rels = {f"Query {k}": v for k, v in rels.items()}

In [82]:
def evaluate_all_metrics(all_ranked_docs, all_relevant_docs, k_list=[5, 10], dcg_cutoff=20):

    metrics_dict = {}
    
    for query_name in all_ranked_docs:
        raw_results = all_ranked_docs[query_name]

        # If raw_results is a dict of {doc_id: score}, sort by score descending
        if isinstance(raw_results, dict):
            ranked_docs = sorted(raw_results, key=raw_results.get, reverse=True)
        else:
            ranked_docs = list(raw_results)
            
        relevant_docs = all_relevant_docs.get(query_name, [])

        metrics = {}
        
        # Overall Precision, Recall, F1
        metrics["Precision"] = overall_precision(ranked_docs, relevant_docs)
        metrics["Recall"] = overall_recall(ranked_docs, relevant_docs)
        metrics["F1"] = overall_f1(ranked_docs, relevant_docs)
        
        # Precision@K, Recall@K, F1@K
        for k in k_list:
            metrics[f"Precision@{k}"] = precision_at_k(ranked_docs, relevant_docs, k)
        
        # R-Precision
        metrics["R-Precision"] = r_precision(ranked_docs, relevant_docs)
        
        # Reciprocal Rank
        metrics["Reciprocal Rank"] = reciprocal_rank(ranked_docs, relevant_docs)
        
        # DCG and nDCG
        metrics[f"DCG@{dcg_cutoff}"] = dcg_at_k(ranked_docs, relevant_docs, dcg_cutoff)
        metrics[f"nDCG@{dcg_cutoff}"] = ndcg_at_k(ranked_docs, relevant_docs, dcg_cutoff)
        
        # Average Precision and Interpolated AP
        metrics["Average Precision"] = average_precision(ranked_docs, relevant_docs)
        metrics["Interpolated AP"] = interpolated_average_precision(ranked_docs, relevant_docs)
        
        # Precision-Recall Curve
        recalls, precisions = precision_recall_curve(ranked_docs, relevant_docs)
        metrics["Precision-Recall Curve"] = (recalls, precisions)
        
        # Interpolated Precision-Recall Curve
        interp_recalls, interp_precisions = interpolated_precision_recall_curve(ranked_docs, relevant_docs)
        metrics["Interpolated Precision-Recall Curve"] = (interp_recalls, interp_precisions)
        
        metrics_dict[query_name] = metrics
        
    # Overall metrics across all queries
    overall = {}
    overall["MAP"] = mean_average_precision(all_ranked_docs, all_relevant_docs)
    overall["Interpolated MAP"] = mean_interpolated_average_precision(all_ranked_docs, all_relevant_docs)
    overall["Gain (%)"] = gain_percent(all_ranked_docs, all_relevant_docs, k=dcg_cutoff)
    
    return metrics_dict, overall


In [83]:
import pandas as pd

def export_evaluation_report(query_metrics, overall_metrics, model_name):
    # --- FILE 1: Per-Query Metrics ---
    # Convert query metrics to DataFrame
    df_queries = pd.DataFrame.from_dict(query_metrics, orient='index')
    
    # Remove the PR Curve data (tuples) so the CSV remains readable/numeric
    cols_to_exclude = ["Precision-Recall Curve", "Interpolated Precision-Recall Curve"]
    df_queries_clean = df_queries.drop(columns=[c for c in cols_to_exclude if c in df_queries.columns])
    
    # Save Query-level results
    df_queries_clean.to_csv(f"metrics_per_query_{model_name}.csv", index_label="Query_ID")
    
    # --- FILE 2: System-Wide Summary ---
    # Convert overall metrics (MAP, etc.) to a vertical table
    df_overall = pd.DataFrame.from_dict(overall_metrics, orient='index', columns=['Value'])
    
    # Save Overall results
    df_overall.to_csv(f"metrics_overall_summary_{model_name}.csv", index_label="Metric")
    
    print("Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'")

In [84]:
import matplotlib.pyplot as plt
import numpy as np

def plot_pr_curves(query_metrics, model_name):
    # 1. Extract all Interpolated Precision arrays (11-point)
    # The data is stored in metrics["Interpolated Precision-Recall Curve"] as (recalls, precisions)
    all_interp_precisions = []
    
    for q in query_metrics:
        _, precs = query_metrics[q]["Interpolated Precision-Recall Curve"]
        all_interp_precisions.append(precs)
    
    # Calculate Mean Interpolated Precision across all queries
    mean_interp_precs = np.mean(all_interp_precisions, axis=0)
    recall_levels = np.linspace(0, 1, 11)

    # 2. Setup the Plot
    plt.figure(figsize=(10, 6))
    
    # Plot the 11-point Interpolated Curve (Step-like)
    plt.step(recall_levels, mean_interp_precs, where='post', label=f'Interpolated PR ({model_name})', linewidth=2, color='blue')
    plt.fill_between(recall_levels, mean_interp_precs, step='post', alpha=0.2, color='blue')

    # Formatting
    plt.title(f'11-Point Interpolated Precision-Recall Curve: {model_name}', fontsize=14)
    plt.xlabel('Recall', fontsize=12)
    plt.ylabel('Precision', fontsize=12)
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()
    
    file_path = f"PR_Curve_{model_name}.png"
    plt.savefig(file_path, dpi=300, bbox_inches='tight')
    plt.close() # Close the plot to free up memory
    
    print(f"Plot saved successfully at: {file_path}")

In [85]:
RSV_LSI_all = {}
doc_ids = M_Weight.columns.tolist()  # La liste des noms de documents (doc_1, doc_2...)

for i, scores in enumerate(sim):
    # 'scores' est le vecteur de similarités pour la requête i
    # On l'associe aux IDs des documents
    query_scores = dict(zip(doc_ids, scores))
    
    # On trie les documents du plus pertinent au moins pertinent
    ranked_docs = sorted(query_scores.items(), key=lambda x: x[1], reverse=True)
    
    # On ne garde que les IDs triés (le format attendu par vos fonctions)
    RSV_LSI_all[f"Query {i+1}"] = [doc for doc, score in ranked_docs]

In [86]:
# 1. Convertir les scores en liste ordonnée de documents
# ranked_lsi = get_ranked_docs(RSV_LSI_all)

In [87]:
rsv_list =[get_ranked_docs(x) for x in [RSV_all1, RSV_all2, RSV_all3, RSV_all4, RSV_BM25_all, RSV_LSI_all]]
method_names = ["Classic BIR – Without learning data", "Classic BIR – With learning data", "Extended BIR – Without learning data", "Extended BIR – With learning data", "BM25", "LSI"]
all_ranked = {}
for rsv, name in zip(rsv_list, method_names):
    ranked = {
        q_id: sorted(docs, key=docs.get, reverse=True) if isinstance(docs, dict) else list(docs)
        for q_id, docs in rsv.items()
    }
    query_metrics, overall_metrics = evaluate_all_metrics(ranked, formatted_rels, k_list=[5, 10], dcg_cutoff=20)
    print(f"\n=== Evaluation Metrics for {name} ===")
    export_evaluation_report(query_metrics, overall_metrics, model_name=name)
    plot_pr_curves(query_metrics, name)
    all_ranked[name] = query_metrics


=== Evaluation Metrics for Classic BIR – Without learning data ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Classic BIR – Without learning data.png

=== Evaluation Metrics for Classic BIR – With learning data ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Classic BIR – With learning data.png

=== Evaluation Metrics for Extended BIR – Without learning data ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Extended BIR – Without learning data.png

=== Evaluation Metrics for Extended BIR – With learning data ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at: PR_Curve_Extended BIR – With learning data.png

=== Evaluation Metrics for BM25 ===
Files created: 'metrics_per_query.csv' and 'metrics_overall_summary.csv'
Plot saved successfully at

In [88]:
def save_rankings_to_csv(rsv_data, method_name, save_dir="rankings"):
    """
    Saves document rankings for each query into a CSV file.
    Compatible with both Score Dictionaries {doc: score} and Ranked Lists [doc1, doc2].
    """
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    
    rows = []
    for query_name, data in rsv_data.items():
        
        # --- LOGIQUE D'ADAPTATION DU FORMAT ---
        if isinstance(data, dict):
            # Format dictionnaire : on trie par score
            ranked_docs = sorted(data.items(), key=lambda x: x[1], reverse=True)
        elif isinstance(data, list):
            # Format liste : c'est déjà trié, on simule un score (ex: rang inverse ou 0)
            # On crée des tuples (doc, score) pour garder la structure uniforme
            ranked_docs = [(doc, "N/A") for doc in data]
        else:
            continue
            
        # --- REMPLISSAGE DES LIGNES ---
        for rank, (doc, score) in enumerate(ranked_docs, 1):
            rows.append({
                "Query": query_name,
                "Rank": rank,
                "Document_ID": doc,
                "Log_Score": score
            })
    
    # Création du CSV
    df = pd.DataFrame(rows)
    clean_name = method_name.replace(" ", "_").lower()
    file_path = os.path.join(save_dir, f"ranking_{clean_name}.csv")
    
    df.to_csv(file_path, index=False)
    print(f"✅ Rankings for {method_name} saved to: {file_path}")

In [105]:
sim_df.T.to_dict()

{'Query 1': {'doc_1': -0.02540317293766524,
  'doc_10': -0.14576689207236776,
  'doc_100': 0.04690447935237089,
  'doc_1000': 0.025968546261064353,
  'doc_1001': 0.04233834139430289,
  'doc_1002': -0.010291624973295908,
  'doc_1003': 0.23986174688656922,
  'doc_1004': 0.011990058345078186,
  'doc_1005': 0.030269843148798603,
  'doc_1006': -0.01559112407437295,
  'doc_1007': -0.030232155158714438,
  'doc_1008': 0.07011768839132623,
  'doc_1009': 0.06395081086348788,
  'doc_101': 0.013680750826816081,
  'doc_1010': 0.010357754235817938,
  'doc_1011': 0.0486172336854789,
  'doc_1012': -0.03236278791310645,
  'doc_1013': 0.028507348995947958,
  'doc_1014': 0.0007277149596498604,
  'doc_1015': 0.010618603330420374,
  'doc_1016': 0.061525208817882336,
  'doc_1017': 0.015204817149328528,
  'doc_1018': -0.02043189715742807,
  'doc_1019': 0.3765392749450642,
  'doc_102': -0.014665482034360853,
  'doc_1020': 0.1636838118596435,
  'doc_1021': 0.11581311578144388,
  'doc_1022': 0.1346612698923054,

In [100]:
print(RSV_all1)

{'Query 1': {'doc_1': 0, 'doc_10': 0, 'doc_100': 1.0974773474870427, 'doc_1000': 0, 'doc_1001': 1.0974773474870427, 'doc_1002': 0, 'doc_1003': 0, 'doc_1004': 0, 'doc_1005': 0, 'doc_1006': 0, 'doc_1007': 0, 'doc_1008': 0, 'doc_1009': 0, 'doc_101': 0, 'doc_1010': 0, 'doc_1011': 0, 'doc_1012': 0, 'doc_1013': 0, 'doc_1014': 0, 'doc_1015': 0, 'doc_1016': 0, 'doc_1017': 0, 'doc_1018': 0, 'doc_1019': 0, 'doc_102': 0, 'doc_1020': 0.9047462468981894, 'doc_1021': 0, 'doc_1022': 0, 'doc_1023': 0, 'doc_1024': 0, 'doc_1025': 0, 'doc_1026': 0, 'doc_1027': 0, 'doc_1028': 0, 'doc_1029': 1.0974773474870427, 'doc_103': 0, 'doc_1030': 0, 'doc_1031': 0, 'doc_1032': 0, 'doc_1033': 0, 'doc_104': 0, 'doc_105': 0, 'doc_106': 0, 'doc_107': 0, 'doc_108': 0.9047462468981894, 'doc_109': 0, 'doc_11': 0.9047462468981894, 'doc_110': 0, 'doc_111': 0, 'doc_112': 1.0974773474870427, 'doc_113': 0, 'doc_114': 1.0974773474870427, 'doc_115': 0, 'doc_116': 0, 'doc_117': 0, 'doc_118': 0, 'doc_119': 0, 'doc_12': 0, 'doc_120':

In [ ]:
for rsv, name in zip([RSV_all1, RSV_all2, RSV_all3, RSV_all4, RSV_BM25_all, sim_df.T.to_dict()], method_names):
    save_rankings_to_csv(rsv, name)

✅ Rankings for Classic BIR – Without learning data saved to: rankings\ranking_classic_bir_–_without_learning_data.csv
✅ Rankings for Classic BIR – With learning data saved to: rankings\ranking_classic_bir_–_with_learning_data.csv
✅ Rankings for Extended BIR – Without learning data saved to: rankings\ranking_extended_bir_–_without_learning_data.csv
✅ Rankings for Extended BIR – With learning data saved to: rankings\ranking_extended_bir_–_with_learning_data.csv
✅ Rankings for BM25 saved to: rankings\ranking_bm25.csv
✅ Rankings for LSI saved to: rankings\ranking_lsi.csv
